<a href="https://colab.research.google.com/github/palswayam5/AI_generated_Images_Classifier/blob/master/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Normalization,Dropout,Add,Embedding,MultiHeadAttention,Input,Flatten
from tensorflow.keras import Model
from tensorflow.image import extract_patches
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')

In [ ]:
df.head()

,labels,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,0,-2.033875,0.978446,-0.142131,-0.177117,-1.470684,1.669562,-0.196530,-0.125239,-0.452284,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,1,-0.348835,0.294815,-0.557577,-2.020773,-1.234715,1.633930,-1.680658,-0.358146,0.166122,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,1,0.113248,-0.607726,-0.947791,0.830851,0.998291,0.498321,-1.493958,0.789572,-1.311018,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,0,1.223321,-0.479048,-1.925789,1.680377,0.021840,-1.453307,0.605559,-0.019024,1.065448,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0,0.160109,0.422684,-0.308029,0.227744,0.432854,0.608348,0.193832,1.035091,-0.538868,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169


In [ ]:
X = df.drop('labels',axis = 1).to_numpy()
y = df['labels'].to_numpy()
X = X.reshape(-1,20,20,3)
y = y.reshape(-1,1)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,shuffle = True)

In [ ]:
print(X_train.shape,y_train.shape,X_val.shape,y_val.shape)

(4200, 20, 20, 3) (4200, 1) (1050, 20, 20, 3) (1050, 1)


Variables

In [1]:
num_layers = 8 #repeating layers in a vision transformer
input_shape = (20,20,3)
patch_size = 4
hidden_units = [1024,2048];
dropout = 0.2
head = 4
dim = 64
num_patches
num_classes = 2

Multi Layer Perceptron

In [ ]:
def mlp(x,hidden_units,dropout_rate):   #multi_layer_perceptron
   for units in hidden_units:
    x =  Dense(units = units)(x)
    x = Dropout(dropout = dropout_rate)(x)
   return x

In [ ]:
def create_patches(image):
  batch_size = tf.shape(image)[0]
  patches = extract_patches(
      images = image,
      size = [1,patch_size,patch_size,1],
      strides = [1,patch_size,patch_size,1],
      padding = "VALID",
      rates = [1,1,1,1],
  )
  patch_dims = patches.shape[-1]
  patches = tf.reshape(patches, [batch_size, -1, patch_dims])
  return patches

In [ ]:
def encode_patches(patches):
  positions = tf.range(start=0, limit=num_patches, delta=1)
  patches = Dense(units = dim)
  positions = Embedding(input_dim = num_patches,ouput_dim = dim)(positions)
  encoded_patches = patches+positions
  return encoded_patches

In [ ]:
def vit_classifier():
  input = Input(shape = input_shape)
  patches = create_patches(input)
  encoded_patches = encode_patches(patches)

  for i in range(num_layers):
    x = Normalization(epsilon = 1e-6)(encoded_patches)
    x1 = MultiHeadAttention(num_head = head,key_dim = dim,dropout = dropout)(x)
    x2 = Add()([x1,encoded_patches])
    x2 = Normalization(epsilon = 1e-6)(x2)
    x3 = mlp(x2,hidden_units,dropout)
    encoded_patches = Add()([x3,x2])
  x = Normalization(epsilon = 1e-6)(encoded_patches)
  x = Flatten()(x)
  x = Dropout(dropout)(x)
  x = mlp(x,hidden_units,dropout)
  y = Dense(units = num_classes,activations = "softmax")(x)
  model = Model(input = input,output = y)
  return model
